In [1]:
# Krishna Thiyagarajan & Abhinav Jain
# Prof. Keene
# Machine Learning - Assignment 2
# Linear Regression

In [2]:
import matplotlib
matplotlib.use('TkAGG')
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
from scipy.stats import multivariate_normal, norm

In [3]:
# All values from book

N = 100
sigma = 0.2
beta = 1/np.power(sigma, 2.0)
alpha = 2
w_0 = -0.3
w_1 = 0.5

In [4]:
# Create sample test data with given mean and variance, and add noise
# Prepare arrays to hold estimated mean and variance for each sample taken

x_n = np.random.uniform (-1, 1, N)
t_n = w_0 + w_1*x_n
noise = np.random.normal (0, sigma, N)
t_n = t_n + noise
mu_n = np.zeros((N, 2))
S_n = np.zeros((N, 2, 2))

# Create a span of the whole grid to be used for plotting prior / posterior
x, y = np.mgrid[-1: 1: 0.01, -1: 1: 0.01]
span = np.dstack((x, y))

fig_reg = plt.figure()

In [5]:
# Set initial mean and variance (from book)
mu_0 = [0, 0]
S_0 = (1.0/alpha) * (np.identity(2))

# Create multivariate normal distribution from the priors and plot them as a contour
# over the span
prior_dist = multivariate_normal(mu_0, S_0)
priorposterior_0 = fig_reg.add_subplot(4, 3, 2)
priorposterior_0.set_xlabel('w0')
priorposterior_0.set_ylabel('w1')

priorposterior_0.contourf(x, y, prior_dist.pdf(span))
priorposterior_0.plot(w_0, w_1, 'w+')

priorposterior_0.set_aspect('equal')

In [6]:
# Create a span for x axis and sample w_0 and w_1 from the distribution
x_dataspace = np.linspace(-1, 1, N)

w_samples_0 = np.random.multivariate_normal(mu_0, S_0, 10)
y_dataspace_0 = np.zeros((10, N))
# For each sample taken, obtain a y value spanning across x axis, and plot the lines
for i in range (10):
    y_dataspace_0[i] = w_samples_0[i, 0] + x_dataspace * w_samples_0[i, 1]

dataspace_0 = fig_reg.add_subplot(4, 3, 3)
dataspace_0.set_xlabel('x')
dataspace_0.set_ylabel('y')
dataspace_0.set_ylim(-1, 1)

for j in range (y_dataspace_0.shape[0]):
    dataspace_0.plot(x_dataspace, y_dataspace_0[j, :])

dataspace_0.set_aspect('equal')

In [7]:
# Linear regression
# Create an initial phi, and add a column for each sample taken
# Calculate S_n and mu_n for n samples
phi = np.array([[1 , x_n[0]]])
for i in range(N):
    if i != 0:
        phi = np.concatenate((phi, np.array([[1, x_n[i]]])), axis=0)
    phiT = phi.T
    S_n_inv = (alpha * np.identity(2)) + (beta * phiT.dot(phi))
    S_n[i] = np.linalg.inv(S_n_inv)
    mu_n[i] = beta * S_n[i].dot(phiT).dot(t_n[0:i+1].T)

In [8]:
# Plot likelihood, prior/posterior and dataspace for the mu_n and S_n calculated above

plot_Ns = [1, 5, N]
for i in plot_Ns:
    
    x_span = np.linspace(1, -1, 100)
    y_span = np.linspace(-1, 1, 100)
    surface = np.zeros((100, 100))
    for j in range (100):
        w_0_likelihood = y_span[j]
        for k in range (100):
            w_1_likelihood = x_span[k]
            mu = w_0_likelihood + w_1_likelihood * x_n[i - 1]
            surface[j, k] = norm(mu, 1/beta).pdf(t_n[i - 1])
    likelihood_n = fig_reg.add_subplot(4, 3, 4 + plot_Ns.index(i) * 3)
    likelihood_n.imshow(surface, cmap='afmhot', interpolation='nearest', extent=[-1, 1, -1, 1])
    likelihood_n.plot(w_0, w_1, 'w+')
    likelihood_n.set_aspect('equal')
    
    
    priorposterior_n = fig_reg.add_subplot(4, 3, 5 + plot_Ns.index(i) * 3)
    rv_n = multivariate_normal(mu_n[i - 1], S_n[i - 1])
    priorposterior_n.plot(w_0, w_1, 'w+')
    priorposterior_n.contourf(x, y, rv_n.pdf(span))
    priorposterior_n.set_aspect('equal')
    
    
    dataspace_n = fig_reg.add_subplot(4, 3, 6 + plot_Ns.index(i) * 3)
    w_samples_n = np.random.multivariate_normal(mu_n[i - 1], S_n[i - 1], 10)
    y_dataspace_n = np.zeros((10, N))
    for j in range (10):
        y_dataspace_n[j] = w_samples_n[j, 0] + x_dataspace * w_samples_n[j, 1]
    for j in range (y_dataspace_n.shape[0]):
        dataspace_n.plot(x_dataspace, y_dataspace_n[j, :])
    for j in range (i):
        dataspace_n.scatter(x_n[0:i], t_n[0:i])
    dataspace_n.set_ylim(-1, 1)
    dataspace_n.set_aspect('equal')

In [9]:
fig_reg.show()